In [15]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os 

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [25]:
# 경로 지정
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')

In [106]:
def crawler(company_code, maxpage):
    
    page = 1 
    
    while page <= int(maxpage): 
    
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
        print(url)
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")
     
 
        
        # 뉴스 제목 
        titles = html.select('.title')
        title_result=[]
        for title in titles: 
            title = title.get_text() 
            title = re.sub('\n','',title)
            title_result.append(title)
 
        # 뉴스 링크
        links = html.select('.title') 
 
        link_result =[]
        for link in links: 
            add = 'https://finance.naver.com' + link.find('a')['href']
            link_result.append(add)

 
        # 뉴스 본문
        links = html.select('.title') 

        content_result =[]
        for link2 in links: 
          add2 = 'https://finance.naver.com' + link2.find('a')['href']
          url2 = add2
          print("url2: "+ url2)
          source_code2 = requests.get(url2).text
          html2 = BeautifulSoup(source_code2, "lxml")

          contents = html2.select('.scr01')
          content_whole = [content.get_text() for content in contents] 
          content_result.append(content_whole)


        # 뉴스 날짜 
        dates = html.select('.date') 
        date_result = [date.get_text() for date in dates] 
 
 
        # 뉴스 매체     
        sources = html.select('.info')
        source_result = [source.get_text() for source in sources] 

        print(title_result)
        print(link_result)
        print(date_result)
        print(source_result)
 
 
        # 변수들 합쳐서 해당 디렉토리에 csv파일로 저장하기 
 
        result= {"날짜" : date_result, "언론사" : source_result, "기사제목" : title_result, "링크" : link_result, "본문" : content_result} 
        df_result = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result.items() ]))

        df_result.to_csv('./crawled_data/'+'page' + str(page) + '.csv', mode='a', encoding='utf-8-sig')             
 
        page += 1 

In [53]:
# 종목 리스트 파일 열기  
# 회사명을 종목코드로 변환 
        
def convert_to_code(company, maxpage):
    
    data = pd.read_csv('./crawl_data/company_list.txt', dtype=str, sep='\t')   # 종목코드 추출 
    company_name = data['회사명']
    keys = [i for i in company_name]    #데이터프레임에서 리스트로 바꾸기 
 
    company_code = data['종목코드']
    values = [j for j in company_code]
 
    dict_result = dict(zip(keys, values))  # 딕셔너리 형태로 회사이름과 종목코드 묶기 
    
    pattern = '[a-zA-Z가-힣]+' 
    
    if bool(re.match(pattern, company)) == True:         # Input에 이름으로 넣었을 때  
        company_code = dict_result.get(str(company))
        crawler(company_code, maxpage)
        print(company_code)
 
    
    else:                                                # Input에 종목코드로 넣었을 때       
        company_code = str(company)      
        # crawler(company_code, maxpage)
        print(company_code)

In [54]:
def main():
    info_main = input("="*50+"\n"+"실시간 뉴스기사 다운받기."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    company = input("종목 이름이나 코드 입력: ") 
    
    maxpage = input("최대 뉴스 페이지 수 입력: ")  
 
    convert_to_code(company, maxpage) 

In [107]:
main()

실시간 뉴스기사 다운받기.
 시작하시려면 Enter를 눌러주세요.
종목 이름이나 코드 입력: 삼성전자
최대 뉴스 페이지 수 입력: 1
https://finance.naver.com/item/news_news.nhn?code=005930&page=1
url2: https://finance.naver.com/item/news_read.nhn?article_id=0004879073&office_id=277&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0004524833&office_id=015&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0004614469&office_id=014&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0001816858&office_id=016&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0004567911&office_id=008&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0001816708&office_id=016&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0004878780&office_id=277&code=005930&page=1&sm=
url2: https://finance.naver.com/item/news_read.nhn?article_id=0003892703&office_id=011&code